In [21]:
import yaml
import networkx as nx
import matplotlib.pyplot as plt
import sys

#Check if the given input (environment file) is empty
def empty_input(d):
    is_empty = False

    if not bool(d):
        is_empty = True

    return is_empty

#Check if the given input (environment file) has duplicate stacks
def has_duplicates(d):
    names = set()
    duplicates = False

    for value in d.values():
        name = value['name']
        if name in names:
            duplicates = True
            break
        names.add(name)

    return duplicates

#Check if a given dependency graph has cycles
def has_cycles(dependency_graph):
    contain_cycles = False
    try:
        cycles = list(nx.find_cycle(dependency_graph))
        if cycles:
            contain_cycles = True
    except:
        contain_cycles = False

    return contain_cycles

#Parse a given stack or environment file
def parse(file):
    d = {}
    # opening a file
    with open(file, 'r') as stream:
        try:
        # Convert yaml document to dictionary
            d = yaml.safe_load(stream)
        except yaml.YAMLError as e:
            print(e)
    
    if empty_input(d) or has_duplicates(d):
        print("Given file is empty or has duplicate entries! Exiting!")
        sys.exit(0)

    return d

def make_dependency_graph(file, plot_graph):
    d = parse(file)

    dependency_graph = nx.DiGraph()

    # Iterate over each entry in the YAML file
    for entry in d:

        name = d[str(entry)]['name']
        dependencies = d[str(entry)]['dependencies']

        # Add node to the graph if it doesn't exist
        if not dependency_graph.has_node(name):
            dependency_graph.add_node(name)

        # Add dependencies to the graph
        for dependency in dependencies:
            if not dependency_graph.has_node(dependency):   
                dependency_graph.add_node(dependency)
            dependency_graph.add_edge(dependency, name)

    if plot_graph:
        # Draw and display the graph
        pos = nx.spring_layout(dependency_graph)
        nx.draw(dependency_graph, pos, with_labels=True, arrows=True)
        plt.show()

    return dependency_graph, d

#Put stacks in topological order
def serialize(file):
    dependency_graph, d = make_dependency_graph(file, False)
    #Check if the graph has cycles. This would be an unwanted instance of the problem
    if has_cycles(dependency_graph):
        print("Detceted cycle in dependency graph. Exiting!!!")

    topological_order = nx.topological_sort(dependency_graph)

    #We create a dictionary with the stacks in topological order
    t_order_dict = {node: list(dependency_graph.predecessors(node)) for node in topological_order}

    return t_order_dict, d

#Check if there are missing dependencies in a given environment file
def verify_and_validate(file, fill_dependencies):
    t_order_dict, dictionary = serialize(file)
    names = [d['name'] for d in dictionary.values()]
    
    #We store the stacks that we already saw in the traversal
    isVisited = {}
    flag=False

    for name, dependencies in t_order_dict.items():
        #If current stack does not have any dependencies
        if not t_order_dict[name] and name in names: 
            #We add it to the visited stacks as a successful insertion (the dependencies of this stack are met)
            isVisited[name] = name
            continue

        for i, dependency in enumerate(dependencies):
            
            if isVisited.get(dependency) is not None or fill_dependencies: 
                if i == len(dependencies) - 1 or fill_dependencies:
                    isVisited[name] = name
            else:
                flag = True
                break
        if flag:
            break

    if flag:
        print("The dependencies are not met!")
    else:
        print("The dependencies are met! Valid environment file!")

    return flag, t_order_dict
    



In [47]:
#Execution of the Required functions

#Uncomment the function that you want to run

# dictionary = parse("stacks.yaml")

# dependency_graph, dictionary = make_dependency_graph("environment_example.yaml", False)

# top_ordered_stacks, dictionary =  serialize("environment_example.yaml")

# is_validated =  verify_and_validate("environment_example.yaml", False)                   #verify and validate

# is_validated, filled_dep_dict =  verify_and_validate("environment_example.yaml", True)   #complete missing dependencies

The dependencies are met! Valid environment file!
